Import tools.

In [1]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from pipelinehelper import PipelineHelper
from sklearn.preprocessing import MaxAbsScaler

In [2]:
import pandas as pd
import numpy as np
from pickle import load
from pickle import dump
pd.set_option("max_columns", None)
pd.set_option('max_rows', None)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score

Bring in dataframe that has race, poverty, and results joined and has Alaska all tidied up.

In [3]:
df = load(open('df_all.pkl', 'rb'))

Look at and drop county and all but one state column for modeling. Drop id.

In [4]:
df = df.drop(['County_x', 'id', 'State_x', 'County_y', 'State_y', 'county'], axis = 1)

In [5]:
state_dummies = pd.get_dummies(df['state'], drop_first = True)
central_outlying = pd.get_dummies(df['central_outlying'], drop_first = True)

In [6]:
df = df.drop(['state', 'central_outlying'], axis = 1)

In [7]:
df = pd.concat([df, state_dummies, central_outlying], axis = 1)

In [8]:
df.Target = df.Target.map({'Trump': 0, 'Clinton': 1})

In [9]:
X = df.drop(['Target'], axis = 1)
y = df.Target

In [10]:
f1_scores = make_scorer(f1_score)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [12]:
# ('lda', LinearDiscriminantAnalysis()),
#         ('qda', QuadraticDiscriminantAnalysis()),
#         ('KNN', KNeighborsClassifier()),
#         ('bayes', GaussianNB()),

In [13]:
pipe = Pipeline([
    ('scaler', PipelineHelper([
        ('std', StandardScaler()),
        ('max', MaxAbsScaler()),
        ('minmax', MinMaxScaler())
    ])),
    ('classifier', PipelineHelper([
        ('svm', svm.SVC())
    ])),
])

In [20]:
params = {
    'scaler__selected_model': pipe.named_steps['scaler'].generate({
        'std__with_mean': [True, False],
        'std__with_std': [True, False],
        'max__copy': [True],  # just for displaying
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'svm__C': [1.0, 2.0, 3.0, 4.0, 10, 20],
        'svm__kernel': ['rbf', 'poly', 'linear'],
        'svm__class_weight': [None, 'balanced']
    })
}

In [21]:
grid = GridSearchCV(pipe, params, scoring= f1_scores)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
# grid.best_estimator_.named_steps['classifier']

In [ ]:
# y_hat_train = cross_val_predict(grid, X_train, y_train)

In [ ]:
# confusion_matrix(y_train, y_hat_train)